In [1]:
import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.keras.layers as tfkl
from tensorflow.keras.models import Sequential


#from google.colab import drive
import numpy as np
import pandas as pd

In this example, we're going to train a [CharRNN](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) on a body of Shakespearian text. Ultimtely, this is an unsuperived learning task. But similar to our previous explorations in unsupervised DL, we will use an unlabeled dataset and create many samples of labeled data that we can use with our familiar supervised loss functions. The result will be a model that has learned the statistical properties of the input text, and can then be considered a "generative" model of language because we can use it to generate synthetic passages of Shakespeare.  

In [3]:
# drive.mount('/content/gdrive/')

In [12]:
file_path = "shakespeare.txt"

with open(file_path,"r") as f:
    text = f.read()

We've loaded our Shakespeare text, let's take a look at a random snippet.

In [13]:
print(text[31600:32000])

 lies i' the second chamber?
  LADY MACBETH. Donalbain.
  MACBETH. This is a sorry sight.           [Looks on his hands.
  LADY MACBETH. A foolish thought, to say a sorry sight.
  MACBETH. There's one did laugh in 's sleep, and one cried,
      "Murther!"
    That they did wake each other. I stood and heard them,
    But they did say their prayers and address'd them
    Again to sleep.
  LADY MACB


We need to convert our text into numeric arrays, the next several blocks accomplish this.

First, we'll create a mapping between characters and their numeric index. We'll also create the reverse mapping, which is useful.

In [15]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 75


Next, we'll create a training set of sub-sequences. Remember, we're trying to train a model to be able to predict the next chracter if it is given several characters of a subsequence. So we will create training pairs where each X is a fixed-length subsequences and each Y is the corresponding next letter in the text.

In [7]:
maxlen = 40 #truncate all the sentence to be 40 at maximum
step = 3
sub_sequences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sub_sequences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sub_sequences))

nb sequences: 38700


In [8]:
k=300
print("(Sequence):\n" + sub_sequences[k])
print("\n(Target Character): \n" + next_chars[k])

(Sequence):
 and other Apparitions
  Lords, Gentleme

(Target Character): 
n


Next we'll create one-hot vectors for our sub-sequences. The tensor we create here will be shaped as (num_sequences x sequence_length x alphabet_size).

In [9]:
X = np.zeros((len(sub_sequences), maxlen, len(chars)), dtype=np.uint8 )
Y = np.zeros((len(sub_sequences), len(chars)), dtype=np.uint8)
for i, seq in enumerate(sub_sequences):
    for t, char in enumerate(seq):
        X[i, t, char_indices[char]] = 1
        Y[i, char_indices[next_chars[i]]] = 1

In [10]:
X[0,0,:]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=uint8)

In [11]:
Y[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

Our RNN model will be quite simple.

In [12]:
char_rnn = Sequential()
char_rnn.add(tfkl.LSTM(128, input_shape=(maxlen, len(chars))))
char_rnn.add(tfkl.Dense(len(chars),activation="softmax"))

In [13]:
char_rnn.compile(loss='categorical_crossentropy', optimizer=tfk.optimizers.RMSprop(lr=0.01))

In [16]:
char_rnn.fit(X[1:7000],Y[1:7000], epochs=10, batch_size=1024)

Train on 6999 samples
Epoch 1/10
6999/6999 [==============================] - 19s 3ms/sample - loss: 2.1359
Epoch 2/10
6999/6999 [==============================] - 20s 3ms/sample - loss: 2.0553
Epoch 3/10
6999/6999 [==============================] - 20s 3ms/sample - loss: 1.9878
Epoch 4/10
6999/6999 [==============================] - 20s 3ms/sample - loss: 1.9455
Epoch 5/10
6999/6999 [==============================] - 22s 3ms/sample - loss: 1.8991
Epoch 6/10
6999/6999 [==============================] - 28s 4ms/sample - loss: 1.8393
Epoch 7/10
6999/6999 [==============================] - 46s 7ms/sample - loss: 1.8051
Epoch 8/10
6999/6999 [==============================] - 37s 5ms/sample - loss: 1.7533
Epoch 9/10
6999/6999 [==============================] - 26s 4ms/sample - loss: 1.7238
Epoch 10/10
6999/6999 [==============================] - 24s 3ms/sample - loss: 1.6703


Once we have a trained model, we can simulate new text by making predictions about the next character and then drawing characters in proportion to the predicted probabilities. And then simple repeat that process over and over, each time drawing the next character.

In [17]:
def draw_char(probs):
    probs = np.asarray(probs).astype('float64')
    if sum(probs) != 1.0:
        probs = probs / np.sum(probs)
    draw = np.random.choice(range(len(probs)) , p=probs)
    return draw

def sample_text(model, sample_length=100):
    start = np.random.randint(0, len(text) - maxlen - 1)
    sequence = text[start: start + maxlen]
  
    x_preds = np.zeros((sample_length, maxlen, len(chars)))
    for i in range(sample_length):
        for t, char in enumerate(sequence[-maxlen:]):
            x_preds[i, t, char_indices[char]] = 1.

        preds = model.predict(np.expand_dims(x_preds[i,:,:], axis=0), verbose=0)[0]
        next_index = draw_char(preds)
        next_char = indices_char[next_index]

        sequence += next_char
    return sequence

In [18]:
sim = sample_text(char_rnn,sample_length=500) 

In [20]:
print(sim)

or
  A Scottish Doctor
  A Sergeant
  A Late, ariove of Yer Durth douch of reath DoLLCANT, y' Dbe chos ver hour a dane-
  S                    Enoun Me.CBETH. Could he Shan lo, my in and de pors
    Nst thean aren sor nigh youk'd hal ve cen,lllin's and hid ach wasd doucalm since.
    In hiol ot lordeld he the kee. The   crow it our the the jane mancot                            Exeem,
 NECON  I' Macite.
   IND WM CBEH. WIis s9
  ROSS. WI TH WIeRD MACLETNG allnDasthe han If  ile ha kerpes and sinven! TEAriin, his hant Den Dourel, erake


Notice that we can do pretty well to learn the typical statistical patterns of this text and then simulate new text that appears to be very similar to legitimate Shakespeare. 

But just a caution - we can also do pretty well with a much simpler method (Markov model): http://nbviewer.jupyter.org/gist/yoavg/d76121dfde2618422139

So the lesson is to try something simple before jumping right in to deep learning.

## Exercise

In this example, we're going to use an RNN for sequence classification. The task we'll set up is to generate a training set of randomized strings, and train our model to detect whether a string contains any vowels.

First, we'll create a training dataset of short randomized character sequences and the corresponding label of whether or not they contain at least one vowel.

In [3]:
import string

In [4]:
def contains_vowels(sequence):
    vowels = ["a", "e", "i", "o", "u"]
    return any([vowel in list(sequence) for vowel in vowels])

In [5]:
contains_vowels("Yunice")

True

In [57]:
sequences = []
labels = []
for i in range(1000):
    char_list = np.random.choice( list(string.ascii_lowercase), size = 5, replace=True)
    seq = "".join(char_list)
    sequences.append(seq)
    labels.append(int(contains_vowels(seq)))

In [58]:
df = pd.DataFrame({"sequence": sequences, "label":labels})

In [59]:
df.head()

,sequence,label
0,cvujz,1
1,gtwkt,0
2,ilojf,1
3,rwmob,1
4,rrfkc,0


Next, set up and train an RNN (of any type) to solve this task. What preprocessing will you need to do first on the raw data in order to prepare it for the network?

In [60]:
# Data Preprocessing
# your code here

In [61]:
OHE = []
for i in range(len(df["sequence"])):
    str_vector = []
    for j in range(len(df["sequence"][i])):
        if contains_vowels(df["sequence"][i][j]) == True:
            str_vector.append(1)
        else:
            str_vector.append(0)
    OHE.append(str_vector)

OHE = np.array(OHE)
label = np.array(df["label"])

In [62]:
OHE[:800]

array([[0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0],
       [1, 0, 1, 0, 0],
       ...,
       [0, 0, 0, 1, 1],
       [1, 0, 0, 1, 1],
       [0, 1, 0, 0, 0]])

In [64]:
label

array([1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1,

In [65]:
# Model setup and training
# your code here

In [72]:
char_rnn = Sequential()
char_rnn.add(tfkl.LSTM(128, input_shape=(800, 5)))
char_rnn.add(tfkl.Dense(26,activation="sigmoid"))

In [73]:
char_rnn.compile(loss='binary_crossentropy', optimizer=tfk.optimizers.RMSprop(lr=0.01))

In [74]:
char_rnn.fit(OHE[:800], label[:800], epochs=10, batch_size=1024)

ValueError: Error when checking input: expected lstm_8_input to have 3 dimensions, but got array with shape (800, 5)